# Import Dependencies 

In [7]:
import math
import requests
import datetime
import json
import urllib
import pandas as pd
import concurrent
from concurrent.futures import ALL_COMPLETED
from requests import Session
from functools import partial
from datetime import date
from six.moves.urllib.parse import urlparse
import re

# Define Helper Functions 

## F&O

In [8]:
def get_data(querystring, rename=0):
    url = "https://www.nseindia.com/api/historical/foCPV"
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}
    cookies = fetch_cookies()
    response = requests.request("GET", url, headers=headers, params=querystring, cookies=cookies)
    data = json.loads(response.text)["data"]
    df = pd.DataFrame(data)
    if rename:
        header = ["id","instrument","symbol","expiry_date","strike_price","option_type","market_type","opening_price","trade_high_price","trade_low_price","closing_price","last_traded_price","prev_cls","settle_price","tot_traded_qty","tot_traded_val","open_int","change_in_oi","market_lot","short_timestamp","long_timestamp","underlying_value"]
        df.columns = header
    return df

def get_adjusted_headers():
    return {
        'Host': 'www.nseindia.com',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:85.0) Gecko/20100101 Firefox/85.0',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'X-Requested-With': 'XMLHttpRequest',
        'DNT': '1',
        'Connection': 'keep-alive',
    }

def fetch_cookies():
    BASE_URL = 'https://www.nseindia.com/'
    response = requests.get(BASE_URL, timeout=30, headers=get_adjusted_headers())
    if response.status_code != requests.codes.ok:
        # logging.error("Fetched url: %s with status code: %s and response from server: %s" % (
        #     BASE_URL, response.status_code, response.content))
        raise ValueError("Please try again in a minute.")
    return response.cookies.get_dict()

## Expiry

In [9]:
idx_exp = {}
vix_exp = {}
stk_exp = {}

def add_dt(instru, dt):
    if not dt.year in instru:
        instru[dt.year] = {}

    if not dt.month in instru[dt.year]:
        instru[dt.year][dt.month] = set()

    instru[dt.year][dt.month].add(dt)

def get_file():
    import http.client
    # fetch_cookies()
    conn = http.client.HTTPSConnection("www1.nseindia.com")
    payload = ""
    headers = {
        'Accept': "*/*",
        'Connection': "keep-alive",
        'Content-Type': "application/x-javascript"
        }
    conn.request("GET", "/products/resources/js/foExp.js", payload, headers)
    res = conn.getresponse()
    data = res.read()
    # print(data.decode("utf-8"))
    return data.decode("utf-8")

def build_dt_dict():
    re_date = re.compile("([0-9]{2}\-[0-9]{2}\-[0-9]{4})")
    lines = get_file()
    for line in lines.split('\n'):
        s = re_date.search(line)
        if s:
            dt = datetime.datetime.strptime(s.group(1), "%d-%m-%Y").date()
            # Start Kludge
            # The list on NSE portal for expiry date has a wrong entry for 20 Sep 2019
            # Handle this oulier use case by ignoring this date and skpping it for processing
            if dt == datetime.datetime(2019, 9, 20).date():
                continue
            # End Kludge
            if line.find('indxExpryDt') > -1:
                try:
                    existing_date = try_to_get_expiry_date(
                        dt.year, dt.month, index=True)
                    if existing_date < dt:
                        add_dt(idx_exp, dt)
                except:
                    add_dt(idx_exp, dt)
            if line.find('stk') > -1:
                try:
                    existing_date = try_to_get_expiry_date(
                        dt.year, dt.month, index=False, stock=False, vix=False)
                    if existing_date < dt:
                        add_dt(stk_exp, dt)
                except:
                    add_dt(stk_exp, dt)
            if line.find('vix') > -1:
                try:
                    existing_date = try_to_get_expiry_date(
                        dt.year, dt.month, index=False, stock=False, vix=True)
                    if existing_date < dt:
                        add_dt(vix_exp, dt)
                except:
                    add_dt(vix_exp, dt)


def try_to_get_expiry_date(year, month, index=True, stock=False, vix=False):
    try:
        if vix and vix_exp:
            return vix_exp[year][month]
        if stock and stk_exp:
            return stk_exp[year][month]
        if index and idx_exp:
            return idx_exp[year][month]
        raise Exception
    except:
        if index:
            name = 'index derivatives'
        elif stock:
            name = 'stock derivatives'
        else:
            name = 'vix derivatives'
        raise ExpiryDateError(
            'No expiry date found in the month of {}-{} for {}'.format(year, month, name))


def get_expiry_date(year, month, index=True, stock=False, vix=False, recursion=0):
    try:
        return try_to_get_expiry_date(year, month, index, stock, vix)
    except:
        if recursion > 1:
            raise
        else:
            pass
    #print("building dictionary")
    build_dt_dict()
    return get_expiry_date(year, month, index, stock, vix, recursion=recursion+1)

# Stock Futures

In [10]:
# # Stock futures (Similarly for index futures, set index = True)
# stock_fut = get_history(symbol="SBIN",
#                         start=date(2015,1,1),
#                         end=date(2015,1,10),
#                         futures=True,
#                         expiry_date=date(2015,1,29))

In [11]:
get_expiry_date(year=2023, month=2, stock=1)

{datetime.date(2023, 2, 23)}

In [12]:
querystring = {
    "from":"20-02-2023",
    "to":"20-04-2023",
    "instrumentType":"FUTSTK",
    "symbol":"ACC",
    "year":"2000",
    "expiryDate":"27-Apr-2023"}

get_data(querystring, rename=1)

,id,instrument,symbol,expiry_date,strike_price,option_type,market_type,opening_price,trade_high_price,trade_low_price,...,prev_cls,settle_price,tot_traded_qty,tot_traded_val,open_int,change_in_oi,market_lot,short_timestamp,long_timestamp,underlying_value
0,642f0450e2684b2e012c5c39,FUTSTK,ACC,27-Apr-2023,.00,XX,N,1842.05,1882.05,1842,...,1848,1872.55,3000,5562412.5,56500,0,250,20-Feb-2023,1850.65,2023-02-19T18:30:00.000Z
1,642f045fe2684b2e013121e3,FUTSTK,ACC,27-Apr-2023,.00,XX,N,1881,1886.3,1850,...,1872.55,1852.7,13250,24675287.5,61500,5000,250,21-Feb-2023,1828.4,2023-02-20T18:30:00.000Z
2,642f0426e2684b2e011e902e,FUTSTK,ACC,27-Apr-2023,.00,XX,N,1834.95,1834.95,1740.35,...,1852.7,1772.15,31250,55707625,65000,3500,250,22-Feb-2023,1752.2,2023-02-21T18:30:00.000Z
3,642f048fe2684b2e014012fc,FUTSTK,ACC,27-Apr-2023,.00,XX,N,1771.45,1778.7,1748.6,...,1772.15,1753,24000,42279750,76250,11250,250,23-Feb-2023,1727.55,2023-02-22T18:30:00.000Z
4,642f047ce2684b2e013a61e6,FUTSTK,ACC,27-Apr-2023,.00,XX,N,1759.35,1771.6,1721.65,...,1753,1752.65,44250,77324537.5,83000,6750,250,24-Feb-2023,1730,2023-02-23T18:30:00.000Z
5,642f0461e2684b2e0131bd78,FUTSTK,ACC,27-Apr-2023,.00,XX,N,1741.9,1747.9,1680.9,...,1752.65,1717.75,73750,125991450,95250,12250,250,27-Feb-2023,1694.8,2023-02-26T18:30:00.000Z
6,642f0452e2684b2e012cf63f,FUTSTK,ACC,27-Apr-2023,.00,XX,N,1703.9,1792.45,1695.95,...,1717.75,1755.05,82750,144870412.5,97750,2500,250,28-Feb-2023,1731.7,2023-02-27T18:30:00.000Z
7,642f0492e2684b2e014101cc,FUTSTK,ACC,27-Apr-2023,.00,XX,N,1768.3,1796.75,1761.25,...,1755.05,1790.7,45750,81532237.5,100750,3000,250,01-Mar-2023,1769.45,2023-02-28T18:30:00.000Z
8,642f043de2684b2e01263e3a,FUTSTK,ACC,27-Apr-2023,.00,XX,N,1784.7,1829.95,1758.05,...,1790.7,1823.25,53750,96894662.5,105250,4500,250,02-Mar-2023,1801.35,2023-03-01T18:30:00.000Z
9,642f047ee2684b2e013ae3ea,FUTSTK,ACC,27-Apr-2023,.00,XX,N,1877.1,1936,1849.6,...,1823.25,1911.35,98000,185664387.5,110000,4750,250,03-Mar-2023,1894.05,2023-03-02T18:30:00.000Z


# Stock Options

In [13]:
# # Stock options (Similarly for index options, set index = True)
# stock_opt = get_history(symbol="SBIN",
#                         start=date(2015,1,1),
#                         end=date(2015,1,10),
#                         option_type="CE",
#                         strike_price=300,
#                         expiry_date=date(2015,1,29))

In [14]:
get_expiry_date(year=2023, month=2, stock=1)

{datetime.date(2023, 2, 23)}

In [15]:
querystring = {
    "from":"20-04-2023",
    "to":"20-05-2023",
    "instrumentType":"OPTSTK",
    "symbol":"ACC",
    "year":"2023",
    "expiryDate":"25-May-2023",
    "optionType":"CE"}

get_data(querystring, rename=0)

,_id,FH_INSTRUMENT,FH_SYMBOL,FH_EXPIRY_DT,FH_STRIKE_PRICE,FH_OPTION_TYPE,FH_MARKET_TYPE,FH_OPENING_PRICE,FH_TRADE_HIGH_PRICE,FH_TRADE_LOW_PRICE,...,FH_PREV_CLS,FH_SETTLE_PRICE,FH_TOT_TRADED_QTY,FH_TOT_TRADED_VAL,FH_OPEN_INT,FH_CHANGE_IN_OI,FH_MARKET_LOT,FH_TIMESTAMP,TIMESTAMP,FH_UNDERLYING_VALUE
0,644149f7bf1903287120027a,OPTSTK,ACC,25-May-2023,2300.00,CE,N,0.00,0.00,0.00,...,18.00,1.25,0,0.00,0,0.00,250,20-Apr-2023,2023-04-19T18:30:00.000Z,1738.1
1,644149f7bf190328711f8920,OPTSTK,ACC,25-May-2023,2280.00,CE,N,0.00,0.00,0.00,...,19.60,1.50,0,0.00,0,0.00,250,20-Apr-2023,2023-04-19T18:30:00.000Z,1738.1
2,644149f7bf1903287120065a,OPTSTK,ACC,25-May-2023,2260.00,CE,N,0.00,0.00,0.00,...,21.35,1.80,0,0.00,0,0.00,250,20-Apr-2023,2023-04-19T18:30:00.000Z,1738.1
3,644149f7bf190328711f5f86,OPTSTK,ACC,25-May-2023,2240.00,CE,N,0.00,0.00,0.00,...,23.25,2.20,0,0.00,0,0.00,250,20-Apr-2023,2023-04-19T18:30:00.000Z,1738.1
4,644149f7bf190328712000b7,OPTSTK,ACC,25-May-2023,2220.00,CE,N,0.00,0.00,0.00,...,25.25,2.65,0,0.00,0,0.00,250,20-Apr-2023,2023-04-19T18:30:00.000Z,1738.1
5,644149f7bf190328711fe626,OPTSTK,ACC,25-May-2023,2200.00,CE,N,0.00,0.00,0.00,...,27.45,3.15,0,0.00,0,0.00,250,20-Apr-2023,2023-04-19T18:30:00.000Z,1738.1
6,644149f7bf190328711fe687,OPTSTK,ACC,25-May-2023,2180.00,CE,N,0.00,0.00,0.00,...,29.80,3.80,0,0.00,0,0.00,250,20-Apr-2023,2023-04-19T18:30:00.000Z,1738.1
7,644149f7bf190328711feb23,OPTSTK,ACC,25-May-2023,2160.00,CE,N,0.00,0.00,0.00,...,32.35,4.50,0,0.00,0,0.00,250,20-Apr-2023,2023-04-19T18:30:00.000Z,1738.1
8,644149f7bf190328712003f8,OPTSTK,ACC,25-May-2023,2140.00,CE,N,0.00,0.00,0.00,...,35.10,5.40,0,0.00,0,0.00,250,20-Apr-2023,2023-04-19T18:30:00.000Z,1738.1
9,644149f7bf190328711fd72b,OPTSTK,ACC,25-May-2023,2120.00,CE,N,0.00,0.00,0.00,...,38.05,6.40,0,0.00,0,0.00,250,20-Apr-2023,2023-04-19T18:30:00.000Z,1738.1


# Index Futures

In [16]:
# nifty_fut = get_history(symbol="NIFTY",
#                         start=date(2015,1,1),
#                         end=date(2015,1,10),
#                         index=True,
#                         futures=True,
#                         expiry_date=date(2015,1,29))

In [17]:
get_expiry_date(year=2023, month=4)

{datetime.date(2023, 4, 3),
 datetime.date(2023, 4, 6),
 datetime.date(2023, 4, 11),
 datetime.date(2023, 4, 13),
 datetime.date(2023, 4, 25),
 datetime.date(2023, 4, 27)}

In [18]:
querystring = {
    "from":"13-05-2023",
    "to":"20-05-2023",
    "instrumentType":"FUTIDX",
    "symbol":"BANKNIFTY",
    "year":"2023"}

get_data(querystring, rename=1)

,id,instrument,symbol,expiry_date,strike_price,option_type,market_type,opening_price,trade_high_price,trade_low_price,...,prev_cls,settle_price,tot_traded_qty,tot_traded_val,open_int,change_in_oi,market_lot,short_timestamp,long_timestamp,underlying_value
0,6463591ff0912e8f612c6be8,FUTIDX,BANKNIFTY,29-Jun-2023,0.00,XX,N,43799.00,44169.65,43749.05,...,43869.35,44096.45,258800,11400061065.00,553250,104525.00,25,15-May-2023,2023-05-14T18:30:00.000Z,NaN
1,6463591ff0912e8f612c621d,FUTIDX,BANKNIFTY,27-Jul-2023,0.00,XX,N,43941.95,44220.40,43872.35,...,43942.00,44155.75,39420,1738749627.75,91980,10875.00,15,15-May-2023,2023-05-14T18:30:00.000Z,NaN
2,64635920f0912e8f612ccfa3,FUTIDX,BANKNIFTY,25-May-2023,0.00,XX,N,43757.20,44075.00,43630.05,...,43757.15,43999.80,2163575,95033988247.50,2321975,79275.00,25,15-May-2023,2023-05-14T18:30:00.000Z,NaN
3,646384d0f0912e8f61325d8c,FUTIDX,BANKNIFTY,29-Jun-2023,0.00,XX,N,44134.95,44194.00,43938.00,...,44096.45,43994.60,376375,16597023253.75,732125,178875.00,25,16-May-2023,2023-05-15T18:30:00.000Z,43903.7
4,646384cff0912e8f6132479c,FUTIDX,BANKNIFTY,27-Jul-2023,0.00,XX,N,44200.00,44235.00,44019.95,...,44155.75,44057.55,32175,1420053987.00,95580,3600.00,15,16-May-2023,2023-05-15T18:30:00.000Z,43903.7
5,646384cff0912e8f61324a2c,FUTIDX,BANKNIFTY,25-May-2023,0.00,XX,N,44050.05,44114.70,43842.90,...,43999.80,43903.15,2062775,90741731612.50,2249675,-72300.00,25,16-May-2023,2023-05-15T18:30:00.000Z,43903.7
6,6464d3e8f0912e8f6153883d,FUTIDX,BANKNIFTY,29-Jun-2023,0.00,XX,N,43946.85,44039.95,43571.90,...,43994.60,43823.90,447925,19615250076.25,864950,132825.00,25,17-May-2023,2023-05-16T18:30:00.000Z,43698.7
7,6464d3e8f0912e8f6153c187,FUTIDX,BANKNIFTY,27-Jul-2023,0.00,XX,N,44000.00,44092.00,43662.85,...,44057.55,43895.95,51585,2261439234.75,88650,-6930.00,15,17-May-2023,2023-05-16T18:30:00.000Z,43698.7
8,6464d3e8f0912e8f61538785,FUTIDX,BANKNIFTY,25-May-2023,0.00,XX,N,43880.60,43955.00,43465.00,...,43903.15,43721.15,2968150,129699410541.25,2080350,-169325.00,25,17-May-2023,2023-05-16T18:30:00.000Z,43698.7
9,6466327ff0912e8f61763882,FUTIDX,BANKNIFTY,29-Jun-2023,0.00,XX,N,44010.10,44134.05,43814.60,...,43823.90,43869.25,426775,18781396915.00,969350,104400.00,25,18-May-2023,2023-05-17T18:30:00.000Z,43752.3


# Index Options

In [19]:
# nifty_opt = get_history(symbol="NIFTY",
#                         start=date(2015,1,1),
#                         end=date(2015,1,10),
#                         index=True,
#                         option_type='CE',
#                         strike_price=8200,
#                         expiry_date=date(2015,1,29))

In [20]:
get_expiry_date(year=2023, month=4)

{datetime.date(2023, 4, 3),
 datetime.date(2023, 4, 6),
 datetime.date(2023, 4, 11),
 datetime.date(2023, 4, 13),
 datetime.date(2023, 4, 25),
 datetime.date(2023, 4, 27)}

In [21]:
querystring = {
    "from":"13-05-2023",
    "to":"20-05-2023",
    "instrumentType":"OPTIDX",
    "symbol":"BANKNIFTY",
    "year":"2023",
    "optionType":"CE"}

get_data(querystring, rename=0)

,_id,FH_INSTRUMENT,FH_SYMBOL,FH_EXPIRY_DT,FH_STRIKE_PRICE,FH_OPTION_TYPE,FH_MARKET_TYPE,FH_OPENING_PRICE,FH_TRADE_HIGH_PRICE,FH_TRADE_LOW_PRICE,...,FH_PREV_CLS,FH_SETTLE_PRICE,FH_TOT_TRADED_QTY,FH_TOT_TRADED_VAL,FH_OPEN_INT,FH_CHANGE_IN_OI,FH_MARKET_LOT,FH_TIMESTAMP,TIMESTAMP,FH_UNDERLYING_VALUE
0,64635920f0912e8f612cd74c,OPTIDX,BANKNIFTY,29-Jun-2023,51000.00,CE,N,21.40,24.95,17.00,...,21.95,17.80,2275,116068945.00,2900,875.00,25,15-May-2023,2023-05-14T18:30:00.000Z,None
1,64635920f0912e8f612ce596,OPTIDX,BANKNIFTY,29-Jun-2023,50000.00,CE,N,0.00,0.00,0.00,...,98.55,133.20,0,0.00,0,0.00,25,15-May-2023,2023-05-14T18:30:00.000Z,None
2,64635920f0912e8f612cd57f,OPTIDX,BANKNIFTY,29-Jun-2023,49500.00,CE,N,19.15,26.75,19.15,...,18.70,22.55,5925,293419883.75,22950,400.00,25,15-May-2023,2023-05-14T18:30:00.000Z,None
3,6463591ff0912e8f612ca22d,OPTIDX,BANKNIFTY,29-Jun-2023,49000.00,CE,N,0.00,0.00,0.00,...,144.65,222.40,0,0.00,0,0.00,25,15-May-2023,2023-05-14T18:30:00.000Z,None
4,6463591ff0912e8f612c2dd3,OPTIDX,BANKNIFTY,29-Jun-2023,48500.00,CE,N,0.00,0.00,0.00,...,175.00,283.50,0,0.00,25,0.00,25,15-May-2023,2023-05-14T18:30:00.000Z,None
5,6463591ff0912e8f612c9a49,OPTIDX,BANKNIFTY,29-Jun-2023,48000.00,CE,N,32.55,41.50,32.35,...,32.25,38.45,44200,2123267981.25,33400,1050.00,25,15-May-2023,2023-05-14T18:30:00.000Z,None
6,6463591ff0912e8f612c42ff,OPTIDX,BANKNIFTY,29-Jun-2023,47500.00,CE,N,0.00,0.00,0.00,...,189.45,448.45,0,0.00,150,0.00,25,15-May-2023,2023-05-14T18:30:00.000Z,None
7,6463591ff0912e8f612cb648,OPTIDX,BANKNIFTY,29-Jun-2023,47000.00,CE,N,64.55,85.65,62.10,...,64.90,75.80,99925,4704057386.25,41900,-9500.00,25,15-May-2023,2023-05-14T18:30:00.000Z,None
8,6463591ff0912e8f612cad85,OPTIDX,BANKNIFTY,29-Jun-2023,46500.00,CE,N,99.05,131.00,96.55,...,102.55,117.65,73625,3432193106.25,121825,11450.00,25,15-May-2023,2023-05-14T18:30:00.000Z,None
9,6463591ff0912e8f612ca3ee,OPTIDX,BANKNIFTY,29-Jun-2023,46300.00,CE,N,0.00,0.00,0.00,...,524.55,740.65,0,0.00,0,0.00,25,15-May-2023,2023-05-14T18:30:00.000Z,None
